In [1]:
!pip install pycountry

   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB 262.6 kB/s eta 0:00:25
   ---------------------------------------- 0.1/6.3 MB 465.5 kB/s eta 0:00:14
   -- ------------------------------------- 0.5/6.3 MB 2.6 MB/s eta 0:00:03
   -------------------- ------------------- 3.2/6.3 MB 14.6 MB/s eta 0:00:01
   ---------------------------------------  6.3/6.3 MB 25.3 MB/s eta 0:00:01
   ---------------------------------------- 6.3/6.3 MB 22.5 MB/s eta 0:00:00


In [3]:
# Importation des librairies utiles
import pandas as pd
import pycountry
import numpy as np

# 1. CHARGEMENT DES DONNÉES SQL EXPORTÉES
csv_file = '0_Global_data_analyst_raw.csv'
df = pd.read_csv(csv_file)

print(f"Données chargées. Nombre de lignes à traiter : {len(df)}")
print("Aperçu des codes pays bruts:\n", df['company_location'].value_counts().head(5))

# 2. FONCTION DE MAPPING DES CODES PAYS (ISO 3166-1 alpha-2)
# Le dataset utilise des codes à 2 lettres (ex: FR, US).
# Nous voulons le nom complet pour Tableau.

def get_country_name(code):
    try:
        # Recherche le pays par code "alpha-2" (2 lettres)
        return pycountry.countries.get(alpha_2=code).name
    except:
        # Si le code n'est pas trouvé
        return 'Unknown'

# 3. APPLICATION DE LA TRANSFORMATION
# Création d'une nouvelle colonne 'Country Name'
df['country_name'] = df['company_location'].apply(get_country_name)

# Création d'une colonne de code à 3 lettres (souvent nécessaire pour Tableau)
def get_alpha_3(code):
    try:
        return pycountry.countries.get(alpha_2=code).alpha_3
    except:
        return np.nan # Nan pour les pays non trouvés

df['country_code_3'] = df['company_location'].apply(get_alpha_3)


# 4. NETTOYAGE FINAL ET SAUVEGARDE

# Vérification : Les lignes où le pays n'a pas pu être mappé sont mises à jour
print(f"\nNombre de lignes non mappées : {len(df[df['country_name'] == 'Unknown'])}")

# Suppression des lignes non mappées si elles sont peu nombreuses
df = df[df['country_name'] != 'Unknown']

# Afficher les 5 premiers pays pour vérification
print("\n--- Aperçu de l'enrichissement (Top 5) ---")
print(df[['company_location', 'country_name', 'country_code_3']].value_counts().head(5))

# Sauvegarde du nouveau fichier CSV enrichi
output_file = 'data_analyst_global_enriched.csv'
df.to_csv(output_file, index=False)

print(f"\nEnrichissement terminé. Sauvegardé dans '{output_file}'.")

Données chargées. Nombre de lignes à traiter : 23040
Aperçu des codes pays bruts:
 company_location
US    19820
CA     1089
GB      998
FR      151
AU      146
Name: count, dtype: int64

Nombre de lignes non mappées : 0

--- Aperçu de l'enrichissement (Top 5) ---
company_location  country_name    country_code_3
US                United States   USA               19820
CA                Canada          CAN                1089
GB                United Kingdom  GBR                 998
FR                France          FRA                 151
AU                Australia       AUS                 146
Name: count, dtype: int64

Enrichissement terminé. Sauvegardé dans 'data_analyst_global_enriched.csv'.
